In [42]:
import numpy as np
import matplotlib.pyplot as plt

class MLP:
    def __init__(self, layer_sizes, transfer_functions):
        # Check the number of layers
        assert len(layer_sizes) - 1 == len(transfer_functions), "Number of hidden layers and transfer functions must be the same"
        assert len(layer_sizes) > 1 and len(layer_sizes) <= 4, "Number of layers should be between 2 and 4"

        # Set random seed for reproducibility
        np.random.seed(42)

        # Initialize network structure
        self.layer_sizes = layer_sizes
        self.transfer_functions = transfer_functions

        # Initialize weights and biases
        self.weights = [np.random.uniform(-2.0, 2.0, (layer_sizes[i+1], layer_sizes[i])) for i in range(len(layer_sizes)-1)]
        self.biases = [np.random.uniform(-2.0, 2.0, (layer_sizes[i+1], 1)) for i in range(len(layer_sizes)-1)]

    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def __sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def __tanh_derivative(self, x):
        return 1 - x**2
    
    def get_activation_function(self, input, function_name):
        match function_name:
            case 'logistic':
                return self.__sigmoid(input)
            case 'tanh': 
                return np.tanh(input)
            case 'identity': 
                return input
            
    def get_activation_function_gradient(self, input, function_name):
        match function_name:
            case 'logistic':
                return self.__sigmoid_derivative(input)
            case 'tanh': 
                return 1 - self.__tanh_derivative(input)
            case 'identity': 
                return 1

    def forward(self, inputs):
        self.activations = [inputs]
        self.weighted_sums = []
        for i in range(len(self.layer_sizes)-1):
            weighted_sum = np.dot(self.weights[i], self.activations[i]) + self.biases[i]
            activation = self.get_activation_function(weighted_sum, self.transfer_functions[i])
            x = self.transfer_functions[i]
            self.activations.append(activation)

        return self.activations[-1]

    def backward(self, inputs, targets, learning_rates):
        # Backward pass
        x = self.transfer_functions[-1]
        errors = [targets - self.activations[-1]]
        deltas = [errors[-1] * self.get_activation_function_gradient(self.activations[-1], self.transfer_functions[-1])]

        for i in reversed(range(len(self.layer_sizes)-2)):
            error = np.dot(self.weights[i+1].T, deltas[0])
            errors.insert(0, error)
            delta = errors[0] * self.get_activation_function_gradient(self.activations[i+1], self.transfer_functions[i])
            deltas.insert(0, delta)

        # Update weights and biases
        for i in range(len(self.layer_sizes)-1):
            self.weights[i] += learning_rates[i] * np.dot(deltas[i], self.activations[i].T)
            self.biases[i] += learning_rates[i] * deltas[i]

    def mean_squared_error(self, targets):
        return np.mean((targets - self.activations[-1])**2)

    def train(self, inputs, targets, epochs, learning_rates):
        learning_curve = []

        for epoch in range(epochs):
            total_error = 0

            for i in range(len(inputs)):
                input_data = np.array(inputs[i]).reshape(-1, 1)
                target_data = np.array(targets[i]).reshape(-1, 1)

                # Forward pass
                self.forward(input_data)

                # Calculate error
                error = self.mean_squared_error(target_data)
                total_error += error

                # Backward pass
                self.backward(input_data, target_data, learning_rates)

            # Calculate and store mean error for the epoch
            mean_error = total_error / len(inputs)
            learning_curve.append(mean_error)

            # Print progress
            print(f"Epoch {epoch+1}/{epochs}, Mean Squared Error: {mean_error}")

        # Save learning curve to a file
        with open("learning_curve.txt", "w") as lc_file:
            lc_file.write("\n".join(map(str, learning_curve)))

    def test(self, test_inputs, test_targets):
        predictions = self.predict(test_inputs)
        test_error = np.mean((test_targets - predictions)**2)

        print(f"\nTest Mean Squared Error: {test_error}")

    def predict(self, inputs):
        predictions = []
        for i in range(len(inputs)):
            input_data = np.array(inputs[i]).reshape(-1, 1)
            output = self.forward(input_data)
            predictions.append(output.flatten())
        return predictions
    
def load_data(path):
    with open(path, 'r') as file:
        file.readline()

        parameters = file.readline().split()

        P, N, M = map(lambda x: int(x.split('=')[1]), parameters[1:])

        data = np.loadtxt(file, dtype=np.dtype(np.float64))
    #data = np.subtract(data, 0.1) #for some reason, 0.1 is added to all values in the file?
    # Extract input and output matrices
    input_matrix = data[:, :N]
    output_matrix = data[:, N:]

    return input_matrix, output_matrix, P, N, M

# Example usage:
if __name__ == "__main__":
    # Set network parameters
    path = "PA-A_training_data_03.txt"
    test_path = "test_data.txt"
    X_test, Y_test, _, _, _ = load_data(path=test_path)
    X_train, Y_train, P, N, M = load_data(path=path)

    layer_sizes = [N, 100, 20, M]  # Number of neurons in each layer
    transfer_functions = ['logistic', 'logistic', 'tanh']  # Transfer function for each layer
    learning_rates = [0.01, 0.005, 0.001]
    # Create MLP
    mlp = MLP(layer_sizes, transfer_functions)

    # Train the MLP
    mlp.train(X_train, Y_train, epochs=100000, learning_rates = learning_rates)

    # Test the trained network
    mlp.test(X_test, Y_test)

    # Plot learning curve
    learning_curve = np.loadtxt("learning_curve.txt")
    plt.plot(learning_curve)
    plt.title('Learning Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Squared Error')
    plt.show()
